In [ ]:
import os
import urllib.request
import librosa
import soundfile
import numpy as np
from flask import Flask, flash, request, redirect, render_template
from werkzeug.utils import secure_filename
import pickle
import pandas as pd

with open(f'model/pred.pkl', 'rb') as f:
    model = pickle.load(f)

UPLOAD_FOLDER = './uploads'

app = Flask(__name__)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

ALLOWED_EXTENSIONS = set(['wav'])

def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        return result


def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
	
@app.route('/')
def upload_form():
	return render_template('upload.html')

@app.route('/', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No file selected for uploading')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            myfile = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            flash('File successfully uploaded')
            feat = extract_feature(myfile, mfcc=True, chroma=True, mel=True)
            feat = feat.reshape(1,-1)
            prediction = model.predict(feat)
            return  render_template('upload.html', result=prediction)
        else:
            flash('Allowed file type is just wav for now')
            return redirect(request.url)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2020 07:55:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:55:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:55:14] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:55:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:55:20] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:55:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:55:26] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:55:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:57:22] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:57:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:57:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:57:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:57:34] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 07:57:34] "GET / HTTP/1.1" 200 -
